# Navigator 에이전트 구축

앞선 실습에서 우리는 `Browser-use` 에이전트가 자연어 명령 하나만으로 브라우저를 직접 조종하는 마법을 경험했습니다.
하지만 매번 브라우저를 새로 실행하고, 이전 대화를 전혀 기억하지 못한다면 실제 업무에서 쓰기엔 한계가 있겠죠?
이번 실습에서는 한 단계 더 나아가, **대화의 맥락을 기억하고 사용자와 자연스럽게 소통하는 'AI 웹 브라우징 어시스턴트'**, 즉 **Navigator 에이전트**를 직접 설계하고 구축합니다.
### 핵심 아이디어: Agent as Tool 패턴
이번 실습의 핵심은 **`Agent as Tool`** 패턴입니다. `Browser-use` 에이전트를 독립 실행 단위로 두는 것이 아니라, LangChain의 `@tool` 데코레이터로 감싸 **하나의 '도구(Tool)'로 전환**합니다.
그리고 이 도구를 더 똑똑한 상위 에이전트에게 쥐여주는 것입니다. 브라우저 조작이라는 '손발'과 대화·추론이라는 '머리'를 분리하는 것이 핵심입니다.

.env 파일을 로드하여 환경변수에 등록합니다. 노트북을 위한 실행 경로를 세팅합니다.

In [1]:
import os, sys
from dotenv import load_dotenv

load_dotenv(override=True)
#load_dotenv("/mnt/d/agent_research/github-repos/home/AAWS_project/.env", override=True)

# PROJECT_ROOT를 .env에서 읽기 (없으면 현재 디렉토리)
project_root = os.getenv("PROJECT_ROOT", os.getcwd())

# 프로젝트 루트가 유효하지 않으면, 현재 위치에서 상위로 찾기
if not os.path.exists(os.path.join(project_root, "app")):
    # 상위 디렉토리 탐색
    current = os.getcwd()
    for _ in range(5):
        if os.path.exists(os.path.join(current, "app")):
            project_root = current
            break
        current = os.path.dirname(current)

# Working Directory 설정
os.chdir(project_root)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import app

print(f"✅ Project Root: {project_root}")
print(f"✅ Working Directory: {os.getcwd()}")

✅ Project Root: /workspaces/codespaces-jupyter/AAWS_project
✅ Working Directory: /workspaces/codespaces-jupyter/AAWS_project


In [2]:
import asyncio
from browser_use import Agent, Browser, ChatGoogle, ChatOpenAI

import nest_asyncio

# 주피터 노트북 환경에서 비동기 루프 중복 실행 오류를 방지하기 위해 사용합니다.
nest_asyncio.apply()

# 가상 디스플레이(noVNC) 번호를 지정합니다.
# 이 설정이 있어야 GUI 브라우저가 가상 화면(:1)에 나타납니다.
os.environ["DISPLAY"] = ":1"

In [3]:
os.environ["GOOGLE_API_KEY"]
os.environ["OPENAI_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "develop_Navigator_agent"

In [6]:
# ==========================================
# 1. 💡 런타임 공유 Context Data
# ==========================================
from dataclasses import dataclass
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langchain.agents import create_agent
from langgraph.store.memory import InMemoryStore
from langgraph.checkpoint.memory import InMemorySaver
from browser_use import Agent, ChatGoogle, ChatOpenAI

@dataclass
class WebAssistantContext:
    # 사용자에 대한 기본 설정값 등을 넣을 수 있는 빈 컨텍스트로 둡니다.
    pass 

# ==========================================
# 2. 💡 범용 웹 탐색 도구 (Browser-use 기반)
# ==========================================
@tool
async def browse_web(instruction: str) -> str:
    """
    주어진 지시사항에 따라 웹 브라우저를 직접 조작하고 결과를 반환합니다.
    검색, 클릭, 텍스트 읽기, 웹사이트 분석 등 사용자가 요청한 웹 관련 작업을 수행할 때 사용하세요.
    
    Args:
        instruction: 브라우저가 수행해야 할 구체적인 행동 지시문 (예: '구글에서 2024년 올림픽 결과 검색해줘')
    """
    print(f"\n🌐 [Browser Tool] 행동 개시: {instruction}")
    
    bu_llm = ChatGoogle(model="gemini-flash-latest")
    #bu_llm = ChatOpenAI(model="gpt-5-mini-2025-08-07")
    
    # 전달받은 instruction을 그대로 Browser-use의 task로 전달합니다.
    agent = Agent(task=instruction, llm=bu_llm)
    history = await agent.run(max_steps=10)
    
    result_text = history.final_result()
    if not result_text:
        return "브라우저 조작을 시도했으나 명확한 결과를 얻지 못했습니다. 다른 명령으로 재시도해보세요."
    
    return result_text

# ==========================================
# 3. 💡 Web Assistant Agent (create_agent) 초기화
# ==========================================
# 멀티턴 대화를 위해 온도를 살짝 높여 자연스럽게 만듭니다 (0.1 -> 0.7)
model = init_chat_model("google_genai:gemini-flash-latest", temperature=0.7) 
store = InMemoryStore()
checkpointer = InMemorySaver()

SYSTEM_PROMPT = """
당신은 사용자와 대화하며 인터넷을 자유롭게 탐색하고 정보를 분석해주는 'AI 웹 브라우징 어시스턴트'입니다.

[역할 및 지침]
1. 당신은 `browse_web` 도구를 사용하여 스스로 웹 브라우저를 제어할 수 있습니다.
2. 사용자가 특정 사이트의 정보를 묻거나, 검색을 요청하거나, 웹 요약을 요구하면 적극적으로 도구를 사용하세요.
3. 도구를 사용할 때는 브라우저 에이전트(도구)가 명확하게 이해할 수 있도록 '구체적인 행동 명령(instruction)'을 작성하여 전달해야 합니다.
   - 나쁜 예: "네이버 뉴스 스크랩"
   - 좋은 예: "https://news.naver.com 에 접속해서 IT/과학 탭의 가장 위에 있는 기사 3개의 제목과 요약을 가져와"
4. 사용자와 자연스러운 멀티턴 대화를 이어나가며, 질문의 의도가 모호하다면 웹을 탐색하기 전에 되물어보세요.
5. 수집된 웹 탐색 결과를 단순히 전달하지 말고, 사용자의 문맥에 맞춰 요약하고 통찰력 있는 언어로 답변하세요.
"""

web_assistant_agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    context_schema=WebAssistantContext,
    tools=[browse_web],
    store=store,
    checkpointer=checkpointer)

In [7]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "my_browsing_session_1"}}
context = WebAssistantContext()

# 1번째 턴 (웹 탐색 유도)
query = "다나와 홈페이지에 접속해서 리뷰가 가장 많고 잘나가는 게이밍 마우스 상위 3개를 찾아봐."
print(f"👤 User: {query}")

response = await web_assistant_agent.ainvoke(
        {"messages": [HumanMessage(query)]},
        config=config,
        context=context
    )

print(f"\n🤖 Assistant: {response['messages'][-1].content}\n")
print("-" * 50)

👤 User: 다나와 홈페이지에 접속해서 리뷰가 가장 많고 잘나가는 게이밍 마우스 상위 3개를 찾아봐.


INFO     [service] Using anonymized telemetry, see https://docs.browser-use.com/development/monitoring/telemetry.
INFO     [Agent] 🔗 Found URL in task: https://danawa.com, adding as initial action...



🌐 [Browser Tool] 행동 개시: 다나와(danawa.com)에 접속해서 '게이밍 마우스'를 검색하고, 인기순 및 리뷰 많은 순으로 정렬하여 상위 3개 제품의 이름, 가격, 그리고 특징을 요약해서 알려줘.


INFO     [Agent] 🎯 Task: 다나와(danawa.com)에 접속해서 '게이밍 마우스'를 검색하고, 인기순 및 리뷰 많은 순으로 정렬하여 상위 3개 제품의 이름, 가격, 그리고 특징을 요약해서 알려줘.
INFO     [Agent] 📦 Newer version available: 0.12.0 (current: 0.11.13). Upgrade with: uv add browser-use==0.12.0
INFO     [Agent] Starting a browser-use agent with version 0.11.13, with provider=google and model=gemini-flash-latest
INFO     [BrowserSession] Setting viewport to 1920x1080 with device scale factor 1.0 whereas original device scale factor was 1.0
INFO     [Agent]   ▶️   navigate: url: https://danawa.com, new_tab: False
INFO     [tools] 🔗 Navigated to https://danawa.com
INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   👍 Eval: The agent successfully navigated to danawa.com. Verdict: Success
INFO     [Agent]   🧠 Memory: I am on the danawa.com homepage and need to search for '게이밍 마우스'.
INFO     [Agent]   🎯 Next goal: Search for '게이밍 마우스' using the search bar.
INFO     [Agent]   ▶️  [1/2] input: index: 105, text: 게이밍 마우스, clear: True
INFO     [


🤖 Assistant: [{'type': 'text', 'text': "다나와에서 현재 가장 잘나가고 리뷰가 많은 게이밍 마우스 상위 3개를 조사해 드립니다.\n\n리뷰가 압도적으로 많은 제품은 보통 보급형 스테디셀러이며, 최근 잘나가는 제품은 고성능 무선 마우스들이 주를 이루고 있습니다. 두 가지 측면을 종합하여 가장 추천할 만한 상위 3개 제품은 다음과 같습니다.\n\n### 1. 로지텍 G102 LIGHTSYNC (리뷰 수 압도적 1위)\n*   **가격:** 약 2~3만 원대\n*   **특징:** 설명이 필요 없는 국민 게이밍 마우스입니다. 다나와 전체 게이밍 마우스 중 리뷰 수가 가장 많으며, 가성비 면에서 압도적입니다.\n*   **장점:** 익숙한 그립감, 준수한 센서 성능, RGB 조명 지원. 피시방에서 가장 흔히 볼 수 있는 모델입니다.\n\n### 2. 로지텍 G PRO X SUPERLIGHT 2 (무선 베스트셀러)\n*   **가격:** 약 19~20만 원대\n*   **특징:** 이른바 '지슈라 2'로 불리는 제품으로, 프로게이머들이 가장 많이 사용하는 모델 중 하나입니다. 인기순위와 매출 상위권을 항상 유지하고 있습니다.\n*   **장점:** 60g 미만의 초경량 설계, 로지텍 최신 HERO 2 센서 탑재, 매우 긴 배터리 수명.\n\n### 3. Razer DeathAdder V3 Pro (데스애더 V3 프로)\n*   **가격:** 약 20만 원대\n*   **특징:** 로지텍과 양대산맥인 레이저(Razer)의 대표 모델입니다. 비대칭형 마우스 중에서 가장 인기 있는 제품으로 리뷰 평점이 매우 높습니다.\n*   **장점:** 63g의 가벼운 무게, 인체공학적 비대칭 디자인, 8K 폴링레이트 지원(동글 별매 시) 등 현존 최고 수준의 스펙을 자랑합니다.\n\n---\n\n**요약하자면:**\n*   **가성비와 입문용**을 찾으신다면 **로지텍 G102**\n*   **대칭형 무선 끝판왕**을 원하시면 **로지텍 G PRO X 

제 지시대로 불명확한 지시의 경우 되묻습니다.

In [8]:
# 2번째 턴
query = "가격과 이름을 알고 싶어."
print(f"👤 User: {query}")

response = await web_assistant_agent.ainvoke(
        {"messages": [HumanMessage(query)]},
        config=config,
        context=context
    )

print(f"\n🤖 Assistant: {response['messages'][-1].content}\n")
print("-" * 50)

👤 User: 가격과 이름을 알고 싶어.

🤖 Assistant: [{'type': 'text', 'text': "다나와에서 현재 '인기순'과 '리뷰가 많은 순'을 종합하여, 실제 구매와 리뷰가 활발한 **게이밍 마우스 상위 3개**의 정확한 명칭과 최저가를 정리해 드립니다. (2024년 5월 다나와 기준)\n\n### 1. 로지텍 G102 LIGHTSYNC (정품)\n*   **가격:** **26,900원**\n*   **특징:** 다나와 내 리뷰 수와 누적 판매량에서 압도적인 1위입니다. '국민 마우스'로 불리며, 가성비를 중시하는 사용자들에게 꾸준히 선택받는 모델입니다.\n\n### 2. 로지텍 G PRO X SUPERLIGHT 2 (지슈라 2)\n*   **가격:** **199,000원**\n*   **특징:** 고성능 무선 마우스 시장의 베스트셀러입니다. 프로게이머들이 가장 선호하는 모델 중 하나로, '인기 상품' 순위 상위권에 항상 위치해 있습니다.\n\n### 3. Razer DeathAdder V3 Pro (데스애더 V3 프로)\n*   **가격:** **214,000원**\n*   **특징:** 레이저(Razer)의 대표적인 비대칭형 무선 마우스입니다. 뛰어난 그립감과 최신 센서 성능 덕분에 많은 리뷰와 높은 평점을 기록하며 인기순위 TOP 3 안에 자주 랭크됩니다.\n\n---\n**추가 참고사항:**\n*   **가장 저렴한 순:** 다나와 검색 결과에서 리뷰가 수만 건에 달하는 가장 저렴한 모델은 **로지텍 G102(약 2.6만 원)**입니다.\n*   **최고 인기 모델:** 현재 실시간 인기 순위 1위는 **CORSAIR 세이버 v2 PRO CF 무선(약 27만 원대)**과 같은 신제품들이 차지하기도 하지만, 꾸준히 잘 나가는 '스테디셀러'는 위 3가지 제품입니다.\n\n원하시는 특정 가격대나 브랜드가 있으시면 더 자세히 찾아봐 드릴 수 있습니다!", 'extras': {'signature': 'EqUfCqIfAb4+9vvL+JDOR1

In [9]:
# 3번째 턴
query = "그 중 제일 싼 건 뭐야?"
print(f"👤 User: {query}")

response = await web_assistant_agent.ainvoke(
        {"messages": [HumanMessage(query)]},
        config=config,
        context=context
    )

print(f"\n🤖 Assistant: {response['messages'][-1].content}\n")
print("-" * 50)


👤 User: 그 중 제일 싼 건 뭐야?

🤖 Assistant: [{'type': 'text', 'text': "방금 말씀드린 상위 3개 제품 중에서 가장 저렴한 제품은 **'로지텍 G102 LIGHTSYNC'**입니다.\n\n*   **제품명:** 로지텍 G102 LIGHTSYNC (정품)\n*   **가격:** 약 **26,900원** (다나와 최저가 기준)\n\n이 제품은 다른 두 모델(지슈라 2, 데스애더 V3 프로)이 20만 원대에 달하는 고가형인 것과 비교하면 **약 7~8배 정도 저렴**합니다. 입문용이나 PC방용으로 가장 많이 쓰이는 '국민 마우스'로, 저렴한 가격 대비 성능과 내구성이 검증되어 리뷰가 압도적으로 많은 모델입니다.", 'extras': {'signature': 'EvgECvUEAb4+9vtEXuMBP38uWnP7JGb5jYjRT2ngUODMCefe4JQ6UvD1uf8v+dYV+nTHBPmpuj4OSvIvvdp2ODHRK6o2aQes6OgsLW8pI5HPwB0Bw8YfOZPP7h+iNx1SkRtvge8ZC09Yhg/dqJSnfg9jiwrCclXNdvK3mGi0KO9P+64QCS1Fnx49w/OFDv0K2rqVn9BeOV0RjXWbhE+PuMQ6O+SqFDT8L4gRt0R3Uj8zq4ridhEiWKTq6tKzzlBCicTXsxJqSbF3LqdDSikqCgKke/CSNR31luLXEwk1YpUJYOheMOvzj9fME1LrYUKdPX/T7IuLg4jfbbzNwC23/UyftodnryMEiNdeBBjbf4y9IgMB/PZ1q41y79r8mvKnSNeKo5nGNb5PhDv2TgGjNkhTif1sEh4ivB7Twc6g4T5Pq6t2TnbUuKjchjdyz+TgVDqC1bflqB0DETXRq0ylO6mo9l70+dRa4eO+fUrS/gc1rqxN/CwvURwhthRmZLllvGrqqJYPLiNuxoNowt/pp312Q47RdKBWLe64wrHlw1XlRcahsOiGzEdqJ6colGNBMlH7LOdV2UIlcabdmFJ4y/1ybklLn40

In [10]:
# 4번째 턴
query = "그 페이지에서 4번째로 인기있는 마우스의 이름은 뭔지 도구를 다시 쓰지 않고 답변할 수 있어?"
print(f"👤 User: {query}")

response = await web_assistant_agent.ainvoke(
        {"messages": [HumanMessage(query)]},
        config=config,
        context=context
    )

print(f"\n🤖 Assistant: {response['messages'][-1].content}\n")
print("-" * 50)

👤 User: 그 페이지에서 4번째로 인기있는 마우스의 이름은 뭔지 도구를 다시 쓰지 않고 답변할 수 있어?

🤖 Assistant: [{'type': 'text', 'text': "아니요, 안타깝게도 **도구를 다시 쓰지 않고는 정확한 답변을 드릴 수 없습니다.**\n\n제가 처음에 다나와에서 정보를 가져올 때 사용자의 요청에 맞춰 **'상위 3개' 제품에 대한 데이터만** 골라서 수집했기 때문입니다. 현재 제 메모리에는 4번째 제품에 대한 정보가 남아 있지 않습니다.\n\n당시 확인했던 상위 3개 제품은 다음과 같았습니다:\n1.  **로지텍 G102** (가장 저렴함)\n2.  **로지텍 G PRO X SUPERLIGHT 2**\n3.  **Razer DeathAdder V3 Pro**\n\n4번째로 인기 있는 마우스가 무엇인지 바로 다시 검색해서 알려드릴까요? 혹은 특정 가격대에서 4번째 제품을 찾으시는 건가요?", 'extras': {'signature': 'EvowCvcwAb4+9vu5b5wYbv2r6nkMzhwsh0VCp3D+hU/izS81JONphxe4P27E/sUlsljbuMZd8oBtR5Jeok9NxALSK4b/QXiKehjDphNDqnmt1Of4BsqlyPPbzoppYEh7/S2ZOEecimLxjjbKG54rIbEpy8lT2gfruJOH0SW9LbePGeAC21Un06m7PvncR0f1w9zcWTKw+jydYIkRGf3NRIVFvK8/Vo2UmDJ8hk9glWk/2k3syvvPidVu9Si8oZMBcTlbZQcrrCBQKGPyHZCfiqEVxRo+6oHHvJw4ir8tuwn6RySJimg1iGEFDFixC6kSb4C+SVUpmEnVJlndPvgl6E1SFppUrBE3Xsfs6VKrllZxCtvpBkFxFu58/tUAgd4LXP0TTYlmXHYxpSHbyKLHI0bkfXs5AN/LKAq7lQPkmNewon6oGEmknYZPPgquwvAHQ/D8Dz3qXa7h85FdeIlggHiYBCKMBqlncEdIq+S0+u/HrZgFVuk55z1n4t5K

### 실습: 단절된 브라우저 탐색 맥락 연결하기

앞선 멀티턴 대화 실험(4번째 턴)에서 한 가지 문제가 발견되었습니다.
*"그 페이지에서 4번째로 인기있는 마우스의 이름은 뭐야?"* 라고 물었을 때, 에이전트가 보고 있던 페이지에서 찾지 않고 **처음부터 다시 다나와를 검색**했습니다.

Navigator 에이전트(상위)는 대화 내용을 기억하고 있지만, `browse_web` 도구가 호출될 때마다 **완전히 새로운 브라우저 세션이 생성**되기 때문에 실제 브라우저는 이전 탐색 맥락(내가 어느 페이지에 있었는지)을 잃어버리는 것입니다.

이 문제를 해결하는 두 가지 방법을 차례대로 실습해 봅시다.

---

#### 🛠️ 방법 1: 결과 텍스트에 방문 URL 함께 반환하기

가장 간단한 방법입니다. `browse_web` 도구가 작업 결과만 넘겨주는 것이 아니라, **마지막으로 도달한 URL**을 함께 적어서 반환합니다. 이렇게 하면 상위 에이전트가 다음 지시를 내릴 때 *"이 URL로 다시 접속해서..."* 라고 명령할 수 있습니다.


In [13]:
# ==========================================
# 실습 1: URL을 반환하는 새로운 도구 정의
# ==========================================
from browser_use import Agent, ChatGoogle

@tool
async def browse_web_url_mode(instruction: str) -> str:
    # tool을 다시 구현하여 봅시다.
    """
    주어진 지시사항에 따라 웹 브라우저를 직접 조작하고 URL을 포함하여 결과를 반환합니다.
    검색, 클릭, 텍스트 읽기, 웹사이트 분석 등 사용자가 요청한 웹 관련 작업을 수행할 때 사용하세요.
    
    Args:
        instruction: 브라우저가 수행해야 할 구체적인 행동 지시문 (예: '구글에서 2024년 올림픽 결과 검색해줘')
    """
    print(f"\n🌐 [Browser Tool] 행동 개시: {instruction}")

    bu_llm = ChatGoogle(model="gemini-flash-latest")
    #bu_llm = ChatOpenAI(model="gpt-5-mini-2025-08-07")

    # 전달받은 instruction을 그대로 Browser-use의 task로 전달합니다.
    agent = Agent(task=instruction, llm=bu_llm)
    history = await agent.run(max_steps=10)

    result_text = history.final_result()
    if not result_text:
        return "브라우저 조작을 시도했으나 명확한 결과를 얻지 못했습니다. 다른 명령으로 재시도해보세요."

    # AgentHistoryList에는 `.url` 속성이 없을 수 있으므로 안전하게 마지막 방문 URL을 추출합니다.
    last_url = None
    try:
        # 우선 제공되는 편의 메서드가 있는지 시도
        if hasattr(history, "urls"):
            urls_list = history.urls() or []
            if urls_list:
                last_url = urls_list[-1]
    except Exception:
        last_url = None

    if not last_url:
        # 대체: 내부 결과(all_results 또는 similar)를 뒤져서 URL을 찾아본다.
        try:
            all_results = getattr(history, "all_results", None) or getattr(history, "results", None) or []
            # 역순으로 살펴보며 "http" 문자열을 포함한 텍스트를 찾아 URL로 간주
            for item in reversed(all_results):
                text_candidates = []
                if hasattr(item, "long_term_memory") and item.long_term_memory:
                    text_candidates.append(item.long_term_memory)
                if hasattr(item, "extracted_content") and item.extracted_content:
                    text_candidates.append(item.extracted_content)
                # 일부 ActionResult의 extracted_content가 리스트일 수 있음
                if hasattr(item, "extracted_content") and isinstance(item.extracted_content, list):
                    text_candidates.extend(item.extracted_content)
                for t in text_candidates:
                    if isinstance(t, str) and ("http://" in t or "https://" in t):
                        # 간단하게 첫 번째 URL-like 토큰을 추출
                        import re
                        m = re.search(r"https?://[^\s,'\)\]]+", t)
                        if m:
                            last_url = m.group(0)
                            break
                if last_url:
                    break
        except Exception:
            last_url = None

    if not last_url:
        last_url = "(URL 확인 불가)"

    return f"URL: {last_url}\n결과: {result_text}"


# 새로운 도구를 장착한 에이전트로 업데이트
web_assistant_agent_v2 = create_agent(
    model=model, system_prompt=SYSTEM_PROMPT, context_schema=WebAssistantContext,
    tools=[browse_web_url_mode], store=store, checkpointer=checkpointer,
)


아래 코드에서 테스트해봅시다.

In [14]:
# 방법 1 테스트 
# URL을 받아서 어떻게 이어가는지 확인합니다.
config_v2 = {"configurable": {"thread_id": "test_session_url"}}

# 1턴: 검색 지시
await web_assistant_agent_v2.ainvoke(
    {"messages": [HumanMessage("네이버 뉴스 IT/과학 메인 페이지에 들어가서 첫 번째 기사 제목을 알려줘.")]},
    config=config_v2
)

print("-" * 50)
# 2턴: 맥락 질문 (불필요한 첫 페이지 검색 없이 전달받은 URL로 직접 가는지 확인)
response = await web_assistant_agent_v2.ainvoke(
    {"messages": [HumanMessage("그 페이지에서 두 번째 기사 제목은 뭐야?")]},
    config=config_v2
)
print(f"\n🤖 Assistant: {response['messages'][-1].content}")


--------------------------------------------------


INFO     [Agent] 🔗 Found URL in task: https://news.naver.com/section/105, adding as initial action...
INFO     [Agent] 🎯 Task: https://news.naver.com/section/105 에 다시 접속해서 두 번째로 노출된 기사의 제목을 알려줘.
INFO     [Agent] 📦 Newer version available: 0.12.0 (current: 0.11.13). Upgrade with: uv add browser-use==0.12.0
INFO     [Agent] Starting a browser-use agent with version 0.11.13, with provider=google and model=gemini-flash-latest



🌐 [Browser Tool] 행동 개시: https://news.naver.com/section/105 에 다시 접속해서 두 번째로 노출된 기사의 제목을 알려줘.


INFO     [BrowserSession] Setting viewport to 1920x1080 with device scale factor 1.0 whereas original device scale factor was 1.0
INFO     [Agent]   ▶️   navigate: url: https://news.naver.com/section/105, new_tab: False
WARNING  [BrowserSession] ⚠️ Page readiness timeout (8.0s, 8174ms) for https://news.naver.com/section/105
INFO     [tools] 🔗 Navigated to https://news.naver.com/section/105
INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   👍 Eval: Successfully navigated to the requested URL and identified the second article title. Verdict: Success
INFO     [Agent]   🧠 Memory: Accessed Naver News IT/Science section. Found the second article title in the headline section.
INFO     [Agent]   🎯 Next goal: Provide the title of the second article to the user.
INFO     [Agent]   ▶️   done: text: 두 번째로 노출된 기사의 제목은 '"휴대폰 보조금 속이면 직접 신고하고 포상금 받으세요"'입니다., success: True, files_to_display: []
INFO     [Agent] 
📄  Final Result: 
두 번째로 노출된 기사의 제목은 '"휴대폰 보조금 속이면 직접 신고하고 포상금 받으세요"'입니다.


I


🤖 Assistant: [{'type': 'text', 'text': '네이버 뉴스 IT/과학 섹션의 두 번째 기사 제목은 다음과 같습니다.\n\n**"휴대폰 보조금 속이면 직접 신고하고 포상금 받으세요"**\n\n추가로 더 궁금한 기사나 내용이 있으신가요?', 'extras': {'signature': 'EuoBCucBAb4+9vs6tSUAbXA5AMlRIkYZ8BYhRc8jsaYQL0YSzEHcSQTOtNvWt7btOfPYJBPC5FldBzxG4kRXvIZtlw0WZ4aHuoxRu6A/WyDWoaE5Cl/Y6QeqUi2ThWV3XbP7eB3GdWNmTus1JQ8hfrm8XUoM/l/iDTuQt0hZKhgmRtWwD/Zn0uDjIxkYFjnp7zksVGgbOA6d5tiPYEyG2TrZAvJd6xm/UlDLPoF5Nn92f+gKD0xgZ61k6jWaF6mXxg3DcT3cATXvtCyTR/LNaZor6m8b7vNrpAn4Zc1X67ASp1A3UzudU0RAMr/I'}}]


---

#### 🛠️ 방법 2: 하나의 브라우저 세션 유지하기 (keep_alive=True)

URL을 넘겨주는 방법은 페이지를 새로고침해야 하므로 스크롤 위치나 동적 상태(로그인 정보)가 초기화된다는 한계가 있습니다. 
가장 완벽한 해결책은 **하나의 브라우저를 열어두고 끄지 않는 것(keep_alive=True)**입니다. 
`browse_web` 도구들이 하나의 브라우저 객체를 닫지 않고 공유해서 사용하도록 만들어 봅시다.


In [19]:
from browser_use import Browser
import os

# DISPLAY 환경변수 확인
print(f"✅ DISPLAY: {os.environ.get('DISPLAY', 'NOT SET')}")

try:
    # 💡 핵심: 세션을 유지하는 공유 브라우저 인스턴스를 하나만 생성합니다.
    print("🚀 브라우저 시작 중...")
    shared_browser = Browser(
        headless=False,
        disable_security=True,
        window_size={'width': 1280, 'height': 720},
        keep_alive=True  # 도구 호출이 끝나도 브라우저가 종료되지 않습니다.
    )
    print("✅ 브라우저 생성 성공!")
except Exception as e:
    print(f"❌ 브라우저 생성 실패: {e}")
    import traceback
    traceback.print_exc()
    raise

@tool
async def browse_web_keep_alive(instruction: str) -> str:
    """
    공유된 브라우저 세션을 사용하여 웹 탐색을 수행하고 결과를 반환합니다.
    여러 턴의 도구 호출에서 브라우저 상태(현재 페이지, 로그인 상태 등)를 유지합니다.
    
    Args:
        instruction: 브라우저가 수행해야 할 구체적인 행동 지시문 (예: '현재 페이지에서 두 번째 링크 클릭해')
    """
    print(f"\n🌐 [Browser Tool - Keep Alive] 행동 개시: {instruction}")
    
    bu_llm = ChatGoogle(model="gemini-flash-latest")
    #bu_llm = ChatOpenAI(model="gpt-5-mini-2025-08-07")
    
    # 공유 브라우저를 전달해서 세션을 유지합니다.
    agent = Agent(task=instruction, llm=bu_llm, browser=shared_browser)
    history = await agent.run(max_steps=10)
    
    result_text = history.final_result()
    if not result_text:
        return "브라우저 조작을 시도했으나 명확한 결과를 얻지 못했습니다. 다른 명령으로 재시도해보세요."
    
    # 현재 페이지 URL 정보 추가 (맥락 유지용)
    last_url = None
    try:
        if hasattr(history, "urls"):
            urls_list = history.urls() or []
            if urls_list:
                last_url = urls_list[-1]
    except Exception:
        last_url = None
    
    if not last_url:
        try:
            all_results = getattr(history, "all_results", None) or getattr(history, "results", None) or []
            for item in reversed(all_results):
                text_candidates = []
                if hasattr(item, "long_term_memory") and item.long_term_memory:
                    text_candidates.append(item.long_term_memory)
                if hasattr(item, "extracted_content") and item.extracted_content:
                    text_candidates.append(item.extracted_content)
                if hasattr(item, "extracted_content") and isinstance(item.extracted_content, list):
                    text_candidates.extend(item.extracted_content)
                for t in text_candidates:
                    if isinstance(t, str) and ("http://" in t or "https://" in t):
                        import re
                        m = re.search(r"https?://[^\s,'\)\]]+", t)
                        if m:
                            last_url = m.group(0)
                            break
                if last_url:
                    break
        except Exception:
            last_url = None

    url_info = f"현재 위치: {last_url}" if last_url else "현재 위치: (URL 확인 불가)"
    
    return f"{url_info}\n결과: {result_text}"


✅ DISPLAY: :1
🚀 브라우저 시작 중...
✅ 브라우저 생성 성공!


In [20]:
# 방법 2: keep_alive 브라우저를 사용하는 새로운 에이전트 생성
web_assistant_agent_v3 = create_agent(
    model=model, system_prompt=SYSTEM_PROMPT, context_schema=WebAssistantContext,
    tools=[browse_web_keep_alive], store=store, checkpointer=checkpointer,
)

In [21]:
# 방법 2 테스트 
# 같은 페이지에서 탭을 끄지 않고 클릭/스크롤만으로 다음 목표를 찾는지 확인합니다.
config_v3 = {"configurable": {"thread_id": "test_session_keep_alive"}}

# 1턴: 검색 지시
await web_assistant_agent_v3.ainvoke(
    {"messages": [HumanMessage("위키백과에서 '인공지능'을 검색해서 개요 부분을 요약해줘.")]},
    config=config_v3
)

print("-" * 50)
# 2턴: 맥락 질문 (브라우저가 꺼지지 않고 현재 위키백과 페이지 안에서 이어서 탐색하는지 확인)
response = await web_assistant_agent_v3.ainvoke(
    {"messages": [HumanMessage("거기서 역사 섹션의 첫 문단만 그대로 읽어줄래?")]},
    config=config_v3
)
print(f"\n🤖 Assistant: {response['messages'][-1].content}")

# ⚠️ 실습이 끝난 후에는 수동으로 브라우저를 반드시 종료시켜주어야 합니다.
await shared_browser.stop()
print("\n🛑 공유 브라우저 세션 종료 완료")

INFO     [Agent] 🎯 Task: 위키백과(https://ko.wikipedia.org)에서 '인공지능'을 검색하여 해당 문서의 개요(목차 전 도입부) 내용을 확인하고 요약해줘.
INFO     [Agent] 📦 Newer version available: 0.12.0 (current: 0.11.13). Upgrade with: uv add browser-use==0.12.0
INFO     [Agent] Starting a browser-use agent with version 0.11.13, with provider=google and model=gemini-flash-latest



🌐 [Browser Tool - Keep Alive] 행동 개시: 위키백과(https://ko.wikipedia.org)에서 '인공지능'을 검색하여 해당 문서의 개요(목차 전 도입부) 내용을 확인하고 요약해줘.


INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   ❔ Eval: Initial step, no previous action to evaluate.
INFO     [Agent]   🧠 Memory: Starting the task to search for '인공지능' on Korean Wikipedia and summarize its introduction.
INFO     [Agent]   🎯 Next goal: Navigate to the Korean Wikipedia homepage.
INFO     [Agent]   ▶️   navigate: url: https://ko.wikipedia.org, new_tab: False
INFO     [tools] 🔗 Navigated to https://ko.wikipedia.org
INFO     [Agent] 📋 Plan updated with 5 steps
INFO     [Agent] 

INFO     [Agent] 📍 Step 2:
INFO     [Agent]   👍 Eval: Successfully navigated to the Korean Wikipedia homepage. Verdict: Success
INFO     [Agent]   🧠 Memory: Currently on the Korean Wikipedia main page. Next step is to search for '인공지능'.
INFO     [Agent]   🎯 Next goal: Search for '인공지능' using the search bar.
INFO     [Agent]   ▶️  [1/2] input: index: 17, text: 인공지능, clear: True
INFO     [BrowserSession] ⌨️ Typed "인공지능" into element with index 17
INFO     [Agent]   ▶️  [2/2] click: 

--------------------------------------------------


INFO     [Agent] 🔗 Found URL in task: https://ko.wikipedia.org/wiki/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5, adding as initial action...
INFO     [Agent] 🎯 Task: https://ko.wikipedia.org/wiki/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5 페이지에서 '역사' 섹션을 찾아 그 첫 번째 문단의 텍스트를 그대로 읽어줘.
INFO     [Agent] 📦 Newer version available: 0.12.0 (current: 0.11.13). Upgrade with: uv add browser-use==0.12.0
INFO     [Agent] Starting a browser-use agent with version 0.11.13, with provider=google and model=gemini-flash-latest
INFO     [Agent]   ▶️   navigate: url: https://ko.wikipedia.org/wiki/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5, new_tab: False



🌐 [Browser Tool - Keep Alive] 행동 개시: https://ko.wikipedia.org/wiki/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5 페이지에서 '역사' 섹션을 찾아 그 첫 번째 문단의 텍스트를 그대로 읽어줘.


INFO     [tools] 🔗 Navigated to https://ko.wikipedia.org/wiki/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5
INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   👍 Eval: The page loaded successfully. Verdict: Success
INFO     [Agent]   🧠 Memory: Navigated to the '인공지능' (Artificial Intelligence) page on Korean Wikipedia. I am about to click on the '역사' (History) section in the table of contents.
INFO     [Agent]   🎯 Next goal: Click on the '역사' link in the table of contents to jump to the History section.
INFO     [Agent]   ▶️   click: index: 15829
INFO     [tools] 🖱️ Clicked a "2
역사"
INFO     [Agent] 📋 Plan updated with 3 steps
INFO     [backoff] Backing off send_request(...) for 0.6s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='eu.i.posthog.com', port=443): Read timed out. (read timeout=15))
INFO     [Agent] 

INFO     [Agent] 📍 Step 2:
INFO     [Agent]   👍 Eval: Successfully navigated to the '역사' section. Verdict: Success
INFO     [Agent]   🧠 Memory: Navigated to '역사


🤖 Assistant: [{'type': 'text', 'text': '위키백과 \'인공지능\' 문서의 **\'역사\'** 섹션 첫 번째 문단 내용은 다음과 같습니다.\n\n> "상당수 인공지능 연구의 (본래) 목적은 심리학에 대한 실험적인 접근이었고, 언어 지능(linguistic intelligence)이 무엇인지를 밝혀내는 것이 주목표였다(튜링 테스트가 대표적인 예이다)."\n\n요청하신 대로 해당 섹션의 첫 문단을 있는 그대로 전달해 드렸습니다. 더 확인이 필요한 정보가 있으신가요?'}]

🛑 공유 브라우저 세션 종료 완료


### 실습: Navigator 앱 서버 연동하기

이제 우리가 만든 훌륭한 '웹 브라우징 어시스턴트'를 Jupyter 노트북을 벗어나 실제 서비스처럼 띄워봅시다!
미리 만들어둔 채팅 서빙용 앱(`app`)에 **Navigator 에이전트**를 추가합니다.

이 앱은 README.md에 설명된 것처럼 **FastAPI(백엔드 서버)**와 **Streamlit(프론트엔드 UI)**로 구성된 구조입니다.
우리가 만든 Navigator를 이 서버의 새로운 에이전트로 등록하기 위해 다음 세 가지 단계를 거칩니다.

#### 1. 에이전트 도구(Tool) 등록하기
`app` 프로젝트는 에이전트가 사용하는 도구 목록을 [app/tools]에서 관리하고 있습니다. 

새로운 파일을 만들고(예를 들어, `app/tools/browser_user_tool.py`), 브라우저 탐색 도구를 여기에 먼저 추가합시다.

[app/tools/__init__.py] 파일을 열고, 우리의 도구를 임포트해서 네비게이터 에이전트가 사용할 도구 리스트를 만들어봅시다.

#### 2. 에이전트 생성 스크립트 작성 (app/agents/navigator_agent.py)

(app/agents/navigator_agent.py)
app/agents 폴더 아래에 navigator_agent.py 파일을 새로 생성하고, 방금 등록한 도구들(tools_navigator)을 불러와서 에이전트를 조립하는 파일입니다.

#### 3. 서버 라우터에 등록 (app/server.py)

이제 새로 만든 에이전트를 FastAPI 백엔드 서버에 연결합시다. 

app/server.py
 파일을 열고 아래 두 부분을 찾아 코드를 추가합니다.


① Agent Executors Import 구문 (약 40번째 줄 부근)

```
from app.agents.chatbot import agent_executor as chatbot_agent
from app.agents.multimodal_agent import agent_executor as multimodal_agent
# 👇 아래 줄 추가
from app.agents.navigator_agent import agent_executor as navigator_agent
```

② Register Routers 라우터 등록 구문 (약 142번째 줄 부근)

```
app.include_router(create_agent_router(chatbot_agent, "/chatbot", ["Chatbot"]))
app.include_router(create_agent_router(multimodal_agent, "/multimodal_agent", ["Multimodal"]))
# 👇 아래 줄 추가 (API 문서 태그명은 Web Navigator로 지정)
app.include_router(create_agent_router(navigator_agent, "/navigator_agent", ["Web Navigator"]))
```

🎉 4. 실행 및 테스트

README.md의 Usage Guide에 나온 방식과 동일하게, 프로젝트 루트 디렉토리(AAWS_project/)에서 서버와 UI를 실행합니다.


1번 터미널: FastAPI 백엔드 서버 실행
`python app/server.py`


2번 터미널:

- CLI 테스트 툴로 빠른 확인
`python app/client.py`

- 또는 streamlit 프론트엔드 UI 실행
`streamlit run app/ui.py`


CLI 테스트: 혹시 브라우저 챗봇 대신 터미널에서 테스트하려면 python app/client.py를 실행하고 /switch navigator_agent 명령어를 쳐서 에이전트를 변경해 보세요!


브라우저 창의 챗봇(Streamlit UI)에서 Agent를 navigator_agent로 바꾸고 대화해 보세요. 질문을 던지면 vnc 화면에서 브라우저가 자동으로 움직이며 맥락을 유지한 상태로 검색을 수행하는 것을 볼 수 있습니다!